In [1]:
! pip install datasets transformers
! pip install pytorch-lightning
! pip install sacremoses
! pip install simplet5 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 7.0 MB/s 
     |████████████████████████████████| 4.4 MB 43.9 MB/s 
     |████████████████████████████████| 101 kB 7.1 MB/s 
     |████████████████████████████████| 140 kB 47.7 MB/s 
     |████████████████████████████████| 1.1 MB 48.0 MB/s 
     |████████████████████████████████| 212 kB 48.0 MB/s 
     |████████████████████████████████| 596 kB 12.4 MB/s 
     |████████████████████████████████| 127 kB 48.2 MB/s 
     |████████████████████████████████| 6.6 MB 45.7 MB/s 
     |████████████████████████████████| 94 kB 1.1 MB/s 
     |████████████████████████████████| 144 kB 27.6 MB/s 
     |████████████████████████████████| 271 kB 48.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing 

In [ ]:
import pytorch_lightning as pl
import pandas as pd
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorWithPadding,
    T5ForConditionalGeneration,
    AdamW, 
    get_linear_schedule_with_warmup
)
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch
import pytorch_lightning as pl
import tarfile
import pandas as pd
import re
import json
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import torch
import torch.nn as nn
from pytorch_lightning.loggers import WandbLogger
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast as BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
import numpy as np

%matplotlib inline
%config InlineBackend.figure_format='retina'
RANDOM_SEED = 42
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8
pl.seed_everything(RANDOM_SEED)

Global seed set to 42


42

In [ ]:
D = pd.read_csv(r'en-ar.csv')
D.dropna(inplace = True)

In [ ]:
D = D.rename(columns = {'verse_ar':'input', 'verse_eng':'target'})

In [ ]:
train, test = train_test_split(D, test_size=0.3, random_state=0)
print(train.shape)
print(test.shape)

(4365, 2)
(1871, 2)


In [ ]:
train

,input,target
5287,إنا بلوناهم كما بلونا أصحاب الجنة إذ أقسموا لي...,Lo! We have tried them as We tried the owners ...
4203,إذ الأغلال في أعناقهم والسلاسل يسحبون,When carcans are about their necks and chains....
4325,حم,Ha. Mim.
1141,قل لا أملك لنفسي نفعا ولا ضرا إلا ما شاء الله ...,"Say: For myself I have no power to benefit, no..."
233,وإن عزموا الطلاق فإن الله سميع عليم,And if they decide upon divorce (let them reme...
...,...,...
4931,سنفرغ لكم أيه الثقلان,"We shall dispose of you, O ye two dependents (..."
3264,فرددناه إلى أمه كي تقر عينها ولا تحزن ولتعلم أ...,So We restored him to his mother that she migh...
1653,وجاء إخوة يوسف فدخلوا عليه فعرفهم وهم له منكرون,And Joseph’s brethren came and presented thems...
2607,يدعو لمن ضره أقرب من نفعه لبئس المولى ولبئس ا...,He calleth unto him whose harm is nearer than ...


In [ ]:
MAX_TOKEN_COUNT = 64
check_point = 'Helsinki-NLP/opus-mt-ar-en'
tokenizer = AutoTokenizer.from_pretrained(check_point)
# model = AutoModelForSeq2SeqLM.from_pretrained(check_point, return_dict = True)

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/896k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/783k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.03M [00:00<?, ?B/s]

In [ ]:
inp = "انا عبدالرحمن من مصر"
input_ids = tokenizer(inp, return_tensors="pt").input_ids
outputs = model.generate(input_ids=input_ids, num_beams=5, num_return_sequences=3)
print("Generated:", tokenizer.batch_decode(outputs, skip_special_tokens=True))

Generated: ['I am Abdurrahman of Egypt.', 'I am Abdulrahman of Egypt.', 'I am Abdul Rahman of Egypt.']


In [ ]:
class TranslationDataset(Dataset):
  def __init__(
      self,
      data: pd.DataFrame,
      tonenizer: AutoTokenizer,
      max_token_len: int = 128
      ):
    self.data = data
    self.data = self.data.reset_index(drop = True)
    self.tokenizer = tokenizer
    self.max_token_len = max_token_len
  def __len__(self):
    return len(self.data)
  def __getitem__(self, index: int):
    input = self.data.at[index, 'input']
    target = self.data.at[index, 'target']
    model_inputs = self.tokenizer(input, max_length=MAX_TOKEN_COUNT, padding='max_length')
    
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = self.tokenizer(target, max_length=MAX_TOKEN_COUNT, padding = 'max_length')

    model_inputs["labels"] = labels["input_ids"]

    # model_inputs['input_ids'] = [np.float64(x)for x in model_inputs['input_ids']]
    # model_inputs['attention_mask'] = [np.float64(x)for x in model_inputs['attention_mask']]
    # model_inputs['labels'] = [np.float64(x)for x in model_inputs['labels']]
    
    return dict(
        input_ids = model_inputs['input_ids'],
        attention_mask = model_inputs['attention_mask'],
        labels = model_inputs['labels']
    )

In [ ]:
train_dataset = TranslationDataset(
  train,
  tokenizer,
  max_token_len=MAX_TOKEN_COUNT
)
sample_item = train_dataset[1]
print(sample_item.keys())
print(sample_item)

dict_keys(['input_ids', 'attention_mask', 'labels'])
{'input_ids': [1360, 1260, 7051, 235, 9, 5805, 17831, 167, 14280, 1532, 125, 37254, 129, 0, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [1530, 2168, 12299, 17, 60, 201, 96, 14407, 17, 6, 22025, 2, 497, 60, 35905, 0, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833,

In [ ]:
test_dataset = TranslationDataset(
  test,
  tokenizer,
  max_token_len=MAX_TOKEN_COUNT
)
sample_item = test_dataset[1]
print(sample_item.keys())
print(sample_item)

dict_keys(['input_ids', 'attention_mask', 'labels'])
{'input_ids': [1210, 546, 5523, 3663, 7055, 167, 13008, 129, 11, 15352, 120, 3615, 7640, 167, 823, 5042, 11, 8019, 0, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [1284, 703, 30667, 5162, 4, 181, 675, 5688, 76, 3, 37991, 45, 181, 127, 54437, 17, 7602, 9758, 4, 0, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 6

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [2]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    "test-trainer",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    learning_rate=2e-5,
    weight_decay=0.01,
)

In [3]:
print(training_args)

TrainingArguments(
_n_gpu=0,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=2e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
log_level=-1,
log_level_replica=-1,
log_on_each_node=True,
logging_dir=test-trainer/runs/Jul2

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 4365
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 273


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=273, training_loss=0.9899229392027243, metrics={'train_runtime': 3956.6904, 'train_samples_per_second': 1.103, 'train_steps_per_second': 0.069, 'total_flos': 81567940608000.0, 'train_loss': 0.9899229392027243, 'epoch': 1.0})

In [ ]:
class TranslationDataModule(pl.LightningDataModule):
  def __init__(self, train_df, test_df, tokenizer, batch_size=8, max_token_len=128):
    super().__init__()
    self.batch_size = batch_size
    self.train_df = train_df
    self.test_df = test_df
    self.tokenizer = tokenizer
    self.max_token_len = max_token_len
  def setup(self, stage=None):
    self.train_dataset = TranslationDataset(
      self.train_df,
      self.tokenizer,
      self.max_token_len
    )
    self.test_dataset = TranslationDataset(
      self.test_df,
      self.tokenizer,
      self.max_token_len
    )
  def train_dataloader(self):
    return DataLoader(
      self.train_dataset,
      batch_size=self.batch_size,
      shuffle=True,
      num_workers=2,
      collate_fn=data_collator
    )
  def val_dataloader(self):
    return DataLoader(
      self.test_dataset,
      batch_size=self.batch_size,
      num_workers=2,
      collate_fn=data_collator
    )
  def test_dataloader(self):
    return DataLoader(
      self.test_dataset,
      batch_size=self.batch_size,
      num_workers=2,
      collate_fn=data_collator
    )

In [ ]:
data_module = TranslationDataModule(train, test, tokenizer, max_token_len = 512)
data_module.setup()
x = data_module.test_dataloader()
next(iter(x))

{'input_ids': tensor([[   68,   326, 11158,  9424, 47542,  1159,  4339,    11,   102, 17548,
          1486,   125,  2049,     0, 62833, 62833, 62833, 62833, 62833, 62833,
         62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833,
         62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833,
         62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833,
         62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833,
         62833, 62833, 62833, 62833],
        [ 1210,   546,  5523,  3663,  7055,   167, 13008,   129,    11, 15352,
           120,  3615,  7640,   167,   823,  5042,    11,  8019,     0, 62833,
         62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833,
         62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833,
         62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833,
         62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833, 62833,


In [ ]:
model.save_pretrained("path/to/model")

Configuration saved in path/to/model/config.json
Model weights saved in path/to/model/pytorch_model.bin


In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("path/to/model")

In [ ]:
inp = "إسمي أحمد انا من برلين"
input_ids = tokenizer(inp, return_tensors="pt").input_ids
outputs = model.generate(input_ids=input_ids, num_beams=5, num_return_sequences=3)
print("Generated:", tokenizer.batch_decode(outputs, skip_special_tokens=True))

Generated: ['My name is Ahmed. I am of Berlin.', 'My name is Ahmed. I am from Berlin.', 'My name is Ahmad. I am of Berlin.']


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
class Translation(pl.LightningModule):
  def __init__(self, n_training_steps=None, n_warmup_steps=None):
    super().__init__()
    self.model = model
    self.n_training_steps = n_training_steps
    self.n_warmup_steps = n_warmup_steps
    self.criterion = nn.CrossEntropyLoss()
  def forward(self, input_ids, attention_mask, labels=None):
    output = self.model.generate(input_ids, attention_mask=attention_mask)
    loss = 0
    labels = [float(label) for label in labels]
    output = [float(o) for o in output]
    if labels is not None:
        loss = self.criterion(output, labels)
    return loss, output
  def training_step(self, batch, batch_idx):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("train_loss", loss, prog_bar=True, logger=True)
    return {"loss": loss, "predictions": outputs, "labels": labels}
  def validation_step(self, batch, batch_idx):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("val_loss", loss, prog_bar=True, logger=True)
    return loss
  def test_step(self, batch, batch_idx):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("test_loss", loss, prog_bar=True, logger=True)
    return loss
  def configure_optimizers(self):
    optimizer = AdamW(self.parameters(), lr=0.00001)
    scheduler = get_linear_schedule_with_warmup(
      optimizer,
      num_warmup_steps=self.n_warmup_steps,
      num_training_steps=self.n_training_steps
    )
    return dict(
      optimizer=optimizer,
      lr_scheduler=dict(
        scheduler=scheduler,
        interval='step'
      )
    )
  def generate(self):
    return model.generate()    

In [ ]:
trainer = pl.Trainer(callbacks=None, 
                     default_root_dir='/content/weights', 
                     gradient_clip_val=0, 
                     process_position=0, 
                     num_nodes=1, 
                     num_processes=1, 
                     gpus=0, 
                     auto_select_gpus=False, 
                     tpu_cores=None, 
                     log_gpu_memory=None, 
                     progress_bar_refresh_rate=30,
                     overfit_batches=0.0, 
                     track_grad_norm=-1, 
                     check_val_every_n_epoch=1, 
                     fast_dev_run=False, 
                     accumulate_grad_batches=1, 
                     max_epochs=1000, 
                     min_epochs=None, 
                     max_steps=None, 
                     min_steps=None, 
                     limit_train_batches=1.0,
                     limit_val_batches=1.0,
                     limit_test_batches=1.0, 
                     limit_predict_batches=1.0, 
                     val_check_interval=1.0, 
                     flush_logs_every_n_steps=100, 
                     log_every_n_steps=50,
                     accelerator=None,
                     sync_batchnorm=False, 
                     precision=32, 
                     weights_summary='top',
                     weights_save_path='/content/weights',
                     num_sanity_val_steps=2,
                     resume_from_checkpoint=None, 
                     profiler=None, 
                     benchmark=False,
                     deterministic=False,
                     reload_dataloaders_every_epoch=False,
                     auto_lr_find=False,
                     replace_sampler_ddp=True,
                     terminate_on_nan=False,
                     auto_scale_batch_size=False, 
                     prepare_data_per_node=True,
                     plugins=None)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/epoch/training_epoch_loop.py:49: LightningDeprecationWarning: Setting `max_steps = None` is deprecated in v1.5 and will no longer be supported in v1.7. Use `max_steps = -1` instead.
  "Setting `max_steps = None` is deprecated in v1.5 and will no longer be supported in v1.7."
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:91: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=30)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:74: Lig

In [ ]:
model = Translation(1000, 10)


In [ ]:
trainer.fit(model, data_module)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/datamodule.py:470: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  f"DataModule.{name} has already been called, so it will not be called again. "
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | MarianMTModel    | 76.8 M
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
76.3 M    Trainable params
524 K     Non-trainable params
76.8 M    Total params
307.335   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

ValueError: ignored